In [ ]:
import numpy as np
from numpy import array,zeros,vstack,repeat,ones,eye,ndarray
from cvxopt import *
import pylab as pl

# So let's talk about the SVM$_\Delta$+

The SVM$_\Delta$+ is a varient on the standard SVM and is written about in [this paper](http://www.jmlr.org/papers/volume16/vapnik15b/vapnik15b.pdf), where instead of just accepting input $X$, and output $Y$, the classifier also has access to extra privileged information $X*$ at training time, but crucially, not at any point past then. So when we come to making predictions, we'll be making them based solely on data looks a lot like $X$. (Quick note, $X*$ doesn't have to be in the same feature space as $X$).

So, some things we're gonna need. Just gonna blast through these at the top because then they're done and out of the way.

We need a data generator as we're going to use toy data (for now)

In [ ]:
# Generates 2D data that is largely linearly separable, but overlaps slightly
def gen_lin_separable_overlap_data(n):
    mean1 = np.array([0, 1])
    mean2 = np.array([1, 0])
    cov = np.array([[1.5, 1.0], [1.0, 1.5]])
    X1 = np.random.multivariate_normal(mean1, cov, (n//2))
    y1 = np.ones(len(X1))
    X2 = np.random.multivariate_normal(mean2, cov, (n//2))
    y2 = np.ones(len(X2)) * -1
    return X1, y1, X2, y2

# Generates 2D data that is linearly separable - no overlap
def gen_lin_separable_data(n):
    # generate training data in the 2-d case
    mean1 = np.array([0, 2])
    mean2 = np.array([2, 0])
    cov = np.array([[0.8, 0.6], [0.6, 0.8]])
    X1 = np.random.multivariate_normal(mean1, cov, (n//2))
    y1 = np.ones(len(X1))
    X2 = np.random.multivariate_normal(mean2, cov, (n//2))
    y2 = np.ones(len(X2)) * -1
    return X1, y1, X2, y2

And we're also going to need a way of visualising this data.

In [ ]:
def plot_margin(X1_train, X2_train, clf):
    def f(x, w, b, c=0):
        # given x, return y such that [x,y] in on the line
        # w.x + b = c
        return (-w[0] * x - b + c) / w[1]

    pl.plot(X1_train[:,0], X1_train[:,1], "ro", label="Class +1")
    pl.plot(X2_train[:,0], X2_train[:,1], "bo", label="Class -1")
    pl.scatter(clf.support_vectors[:,0], clf.support_vectors[:,1], s=100, c="g")

    # w.x + b = 0
    a0 = -4; a1 = f(a0, clf.w, clf.b)
    b0 = 4; b1 = f(b0, clf.w, clf.b)
    pl.plot([a0,b0], [a1,b1], "k")

    # w.x + b = 1
    a0 = -4; a1 = f(a0, clf.w, clf.b, 1)
    b0 = 4; b1 = f(b0, clf.w, clf.b, 1)
    pl.plot([a0,b0], [a1,b1], "k--")

    # w.x + b = -1
    a0 = -4; a1 = f(a0, clf.w, clf.b, -1)
    b0 = 4; b1 = f(b0, clf.w, clf.b, -1)
    pl.plot([a0,b0], [a1,b1], "k--")

    pl.xlabel('x1')
    pl.ylabel('x2')

    pl.legend(numpoints=1)

    pl.axis("tight")
    pl.show()

Ignore this bit, it's so we can have different kernels on our data... It shouldn't make a difference for now.

In [ ]:
class Linear():
    def __call__(self, a, b):
        x = np.array(a)
        y = np.array(b)
        y = np.transpose(y)
        return np.dot(x, y)

class Polynomial():
    def __call__(self, a, b, p=3):
        x = np.array(a)
        y = np.array(b)
        y = np.transpose(y)
        return (1 + np.dot(x, y)) ** p

class Gaussian():
    def __call__(self, a, b, sigma=5.0):
        x = np.array(a)
        y = np.array(b)
        y = np.transpose(y)
        return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

And we're going to create a "problem" class. This is just somewhere where we keep the data that we're training the SVM on, but it also holds some hyperparameters that we might need later on.

In [ ]:
class svm_problem():
    def __init__(self, C=1.0, gamma=1.0, delta=1.0, kernel=Linear()):
        self.C = C
        self.gamma = gamma
        self.delta = delta
        self.kernel = kernel

    def set_variables(self, X, Xstar, Y):
        if(isinstance(X, ndarray)):
            self.X = X
        else:
            self.X = array(X)
        if(isinstance(Xstar, ndarray)):
            self.Xstar = Xstar
        else:
            self.Xstar = array(Xstar)
        if(isinstance(Y, ndarray)):
            self.Y = Y
        else:
            self.Y = array(Y)
        self.num = len(self.X)
        self.dimensions = len(self.X[0])
        self.xi_xj = self.gram_matrix(self.X, self.X)
        self.xstari_xstarj = self.gram_matrix(self.Xstar, self.Xstar)
        self.yi_yj = self.gram_matrix(self.Y, self.Y)
        self.xi_star = zeros(self.num)
        self.xi = zeros(self.num)
        self.zeta = zeros(self.num)

    def gram_matrix(self, X1, X2):
        K = zeros((len(X1), len(X1)))
        for i in range(len(X1)):
            for j in range(len(X1)):
                K[i,j] = self.kernel(X1[i], X2[j])
        return K

Sweet, so what is it that we're solving?

The regular SVM looks like this. We want to
$$\begin{equation}
\begin{aligned}
\min_{\textbf{w},b,\xi} \quad & \frac{1}{2}||\textbf{w}||^2 + C\sum_{i=1}^{\ell}\xi_i\\
\textrm{subject to} & \quad y_i(\textbf{w} \cdot \textbf{x}_i+b)\geq 1 - \xi_i \\
\textrm{and} & \quad \xi_i \geq 0
\end{aligned}
\end{equation}$$
Meaning we want to make the maergin as wide as possible in our classifier, but we're going to be penalised for every breach of that margin. So, breaches are allowed, but they cost.

Turns out this is actually quite hard to solve as the conditions are quite complex. If we change this to it's dual 
$$\begin{equation}
\begin{aligned}
\max_{\alpha} \sum_{i=1}^{\ell}\alpha_i - \frac{1}{2} & \sum_{i,j=1}^{\ell}\alpha_i \alpha_j y_i y_j (\textbf{x}_i \cdot \textbf{x}_j) \\
\textrm{subject to} & \quad 0 < \alpha_i < C \\
\textrm{and} & \quad \sum_{i=1}^{m} \alpha_i y_i = 0
\end{aligned}
\end{equation}$$
We get a convex optimisation problem that can be solved using such convex optimisation tools as CVXOPT.

CVXOPT solves problems in the following format
$$\begin{equation}
\begin{aligned}
\min_{x} \quad & \frac{1}{2}x^TPx+q^Tx \\
\textrm{subject to} \quad & Gx \leq h \\
\textrm{and} \quad & Ax = b
\end{aligned}
\end{equation}$$

which as you can see is sort of the same shape as the dual form of the SVM problem. But instead of minimising over $x$ we want to maximise over $/alpha$. Doing this is easy enough. We just have to do a bit of rearranging, giving us
$$\begin{equation}
\begin{aligned}
\min_{\alpha} & \quad \frac{1}{2} \sum_{i,j=1}^{\ell}\alpha_i (y_i y_j (\textbf{x}_i \cdot \textbf{x}_j)) \alpha_j - \sum_{i=1}^{\ell}\alpha_i \\
\textrm{subject to} & \quad -\alpha_i < 0 \\
\textrm{and} & \quad \alpha_i < C \\
\textrm{and} & \quad \sum_{i=1}^{m} \alpha_i y_i = 0
\end{aligned}
\end{equation}$$

To form P we make a Gram Matrix of $Y$ and a second of $X$ then multiply them together 
$$\begin{equation}
P = 
\begin{pmatrix} y_1y_1(\textbf{x}_1\textbf{x}_1) & \cdots & y_iy_1(\textbf{x}_i\textbf{x}_1)\\
\vdots & \ddots & \vdots \\
y_1y_j(\textbf{x}_1\textbf{x}_j) & \cdots & y_iy_j(\textbf{x}_i\textbf{x}_j)
\end{pmatrix} 
\end{equation}$$

q is the negative identity matrix $$\begin{equation}
q = 
\begin{pmatrix} -1 & 0 & \cdots & 0 \\
0 & -1 & \cdots & 0 \\
\vdots & \cdots & \ddots & \vdots \\
0 & \cdots & 0 & -1
\end{pmatrix} 
\end{equation}$$

G and h are slightly more tricky as we've got two less than statements, but let's break them down and talk about G1 and h1 being $0\leq\alpha_i$ and G2 and h2 being $\alpha_i\leq C$

$\begin{equation}
G1 = 
\begin{pmatrix} -1 & 0 & \cdots & 0 \\
0 & -1 & \cdots & 0 \\
\vdots & \cdots & \ddots & \vdots \\
0 & \cdots & 0 & -1
\end{pmatrix} 
\end{equation}$
$\begin{equation}
h1 = 
\begin{pmatrix} 0_1 \\
0_2 \\
\vdots \\
0_n
\end{pmatrix} 
\end{equation}$

$-1\times\alpha_1\leq 0 \cdots -1\times\alpha_n\leq 0$

$\begin{equation}
G2 = 
\begin{pmatrix} 1 & 0 & \cdots & 0 \\
0 & 1 & \cdots & 0 \\
\vdots & \cdots & \ddots & \vdots \\
0 & \cdots & 0 & 1
\end{pmatrix} 
\end{equation}$
$\begin{equation}
h2 = 
\begin{pmatrix} C_1 \\
C_2 \\
\vdots \\
C_n
\end{pmatrix} 
\end{equation}$
C is a constant, so $C_n$ means the $n^{th}$ repitition of C

Then we stack them so that $\begin{equation}
G = 
\begin{pmatrix} G1 \\
G2
\end{pmatrix} 
\end{equation}$
and $\begin{equation}
h = 
\begin{pmatrix} h1 \\
h2
\end{pmatrix} 
\end{equation}$ Don't think this is some clever notation. I literally mean G1 stacked on top of G2. Same with h.

$\begin{equation}
A = 
\begin{pmatrix} y_1 & y_2 & \cdots & y_n
\end{pmatrix} 
\end{equation}$ $\begin{equation}
b = 
\begin{pmatrix} 0
\end{pmatrix} 
\end{equation}$

Notice that A is a vector, not a column matrix. This is because we want the sum of $\alpha y$ to be 0, not element-wise

So now lets go ahead and put this in a solver. This might be a bit dull, but first we're going to generate some data. $X$ is going to have some overlap, but the privileged data will be separable. As at the moment we're just looking at the regular SVM, we'll ignore the $X*$ data.

In [ ]:
x1, y1, x2, y2 = gen_lin_separable_overlap_data(20)
X = np.vstack((x1,x2))
Y = np.hstack((y1,y2))
x3,y1,x4,y2 = gen_lin_separable_data(20)
Xstar = np.vstack((x3,x4))

prob = svm_problem(C=1.0)
prob.set_variables(X, Xstar, Y)

In [ ]:
x = prob.X
y = prob.Y
C = prob.C

NUM = x.shape[0]
DIM = x.shape[1]

Ky = prob.yi_yj
Kx = prob.xi_xj
K = Ky*Kx
P = matrix(K)
q = matrix(-np.ones((NUM, 1)))
G1 = -np.eye(NUM)
G2 = np.eye(NUM)
G = np.vstack((G1, G2))
G = matrix(G)
h1 = np.zeros(NUM).reshape(-1,1)
h2 = np.repeat(C, NUM).reshape(-1,1)
h = np.vstack((h1, h2))
h = matrix(h)
A = matrix(y.reshape(1, -1))
b = matrix(np.zeros(1))
solvers.options['show_progress'] = False
sol = solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

If you print out alphas, you should get an array of values that are proactiaclly 0, except for the supprt vectors. They're slightly bigger than 0. Depending on the hyperparameters, there should be very few of them.

So having the alphas is great, but we want to extract weights and bias to make a prediction... So lets do that now.

$$\textbf{w} = \sum_{i=1}^{L}\alpha_iy_i\textbf{x}_i$$

In [ ]:
w = np.sum(alphas * y[:, None] * x, axis = 0)

The bias is a bit trickier. We need to get the set of support vectors ($S$) that match $0<\alpha_i\leqC$ then solve

$$b = \frac{\sum_{s=1}^{S}(y_s-\sum_{m=1}^{S}\alpha_my_m(\textbf{x}_m\cdot\textbf{x}_s))}{\textrm{Number of S}}$$

As you might imagine, this is a bit of a faff, but below does the trick

In [ ]:
bacond1 = (alphas > 1e-5)
bdcond2 = (alphas < C)

bcond = np.array([a and b for a, b in zip(bacond1, bdcond2)]).flatten()

yS = y[bcond]
xS = x[bcond]
aS = alphas[bcond]


sumTotal = 0
for s in range(len(yS)):
    innerTotal = 0
    for m in range(len(yS)):
        am = aS[m]
        ym = yS[m]
        xm_xs = prob.kernel(xS[m], xS[s])
        innerTotal += am*ym*xm_xs
    sumTotal += yS[s] - innerTotal

bias = sumTotal/len(yS)

Then we put this in a a dull, but useful class called classifier, just to keep everything neat and out of the way.

In [ ]:
class classifier():

    def __init__(self):
        self.w = 0
        self.b = 0
        self.alphas = []
        self.support_vectors = []

    def predict(self, x):
        return np.sign(np.dot(self.w,x)+self.b)
    
    def f_star(self, x, y): # This won't make sense now, but we come back to it later
        return y*(np.dot(self.w,x)+self.b)

In [ ]:
svm_clf = classifier()
svm_clf.w = w
svm_clf.b = bias
svm_clf.alphas = alphas
svm_clf.support_vectors = prob.X[bacond1.flatten()]

And that's it.... a trained SVM. Shall we take a look at it?

In [ ]:
plot_margin(prob.X[y==1], prob.X[y==-1], svm_clf)

Which is all a bit sexy. Go back to the problem definition and play with the C value. the bigge then number (like 10?), the harder the margin, the lower (say, 0.01), the softer.

BUT WHAT DOES THIS HAVE TO DO WITH SVM$_\Delta$+? Everything. There are two versions, a simplified approach and a non-simplified approach. Let's break them down. But first I'm just going to put the above in a class just so that it's eaiser if we want to use it in the future.

In [ ]:
class SVM():
    def train(self, x, prob : svm_problem):
        x = x
        y = prob.Y
        C = prob.C

        NUM = x.shape[0]
        DIM = x.shape[1]

        K = y[:, None] * x # Yeah, this is a bit different so that it can work on x and x*
        K = np.dot(K, K.T)
        P = matrix(K)
        q = matrix(-np.ones((NUM, 1)))
        G1 = -np.eye(NUM)
        G2 = np.eye(NUM)
        G = np.vstack((G1, G2))
        G = matrix(G)
        h1 = np.zeros(NUM).reshape(-1,1)
        h2 = np.repeat(C, NUM).reshape(-1,1)
        h = np.vstack((h1, h2))
        h = matrix(h)
        A = matrix(y.reshape(1, -1))
        b = matrix(np.zeros(1))
        solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h, A, b)
        alphas = np.array(sol['x'])
        w = np.sum(alphas * y[:, None] * x, axis = 0)
        bacond1 = (alphas > 1e-5)
        bdcond2 = (alphas < C)
        bcond = np.array([a and b for a, b in zip(bacond1, bdcond2)]).flatten()
        yS = y[bcond]
        xS = x[bcond]
        aS = alphas[bcond]
        sumTotal = 0
        for s in range(len(yS)):
            innerTotal = 0
            for m in range(len(yS)):
                am = aS[m]
                ym = yS[m]
                xm_xs = prob.kernel(xS[m], xS[s])
                innerTotal += am*ym*xm_xs
            sumTotal += yS[s] - innerTotal
        bias = sumTotal/len(yS)
        clf = classifier()
        clf.w = w
        clf.b = bias[0]
        clf.alphas = alphas
        clf.support_vectors = x[bacond1.flatten()]
        return clf

## SVM$_\Delta$+ - The simplified approach

The simplified approach uses the SVM we've already written. If you want to see this on the paper we're following, it's [page 2034](http://www.jmlr.org/papers/volume16/vapnik15b/vapnik15b.pdf "Vapnik paper"). Firstly we learn an SVM in the privileged space $X*$

In [ ]:
svm = SVM()
xStar_clf = svm.train(prob.Xstar, prob)

Then we use this to work out a set of slack values in the privileged space $\xi^* = [1-f(x^*)-b^*]_+$. This means we choose the highest value of 0 or the predicted value 

In [ ]:
xi_star = np.zeros(prob.num)
for i in range(prob.num):
    output = (1-xStar_clf.f_star(prob.Xstar[i], prob.Y[i]) - xStar_clf.b)
    xi_star[i] = max(0, output)

So if we were to look at the $X*$ classifier on a graph, we should see that there aren't really any breaches as we've used a linearly sepearble set of data for $X*$, consequently, the values of $\xi^*$ should also all be almost 0 (except the ones that breach the margin).

In [ ]:
plot_margin(prob.Xstar[y==1], prob.Xstar[y==-1], xStar_clf)

In [ ]:
xi_star

we then basically train another SVM. This time though, instead of using exactly the same model as before, we have to do a different one as there are different constraints. Let's look at them. 
$$\begin{equation}
\begin{aligned}
\max_{\alpha} \sum_{i=1}^{\ell}\alpha_i - \frac{1}{2} & \sum_{i,j=1}^{\ell}\alpha_i \alpha_j y_i y_j (\textbf{x}_i \cdot \textbf{x}_j) \\
\textrm{subject to} & \quad \sum_{i=1}^{\ell}\alpha_i\xi^*_i \leq C\sum_{i=1}^{\ell}\xi^*_i \\
\textrm{and} & \quad 0 < \alpha_i < (1+\Delta)C \\
\textrm{and} & \quad \sum_{i=1}^{m} \alpha_i y_i = 0
\end{aligned}
\end{equation}$$

So let's break it down. The main optimisation problem is the same as a regular SVM, so we can use the same P and q as before. The equality constraint is the same, so we have the same A and b. We just need a new G and h. Let's break it down. G1 and h1 give $-\alpha\leq0$, G2 and h2 $\alpha\leq(1+\Delta)C$ and G3 and h3 being $\sum_{i=1}^{\ell}\alpha_i\xi^*_i \leq C\sum_{i=1}^{\ell}\xi^*_i$

$\begin{equation}
G1 = 
\begin{pmatrix} -1 & 0 & \cdots & 0 \\
0 & -1 & \cdots & 0 \\
\vdots & \cdots & \ddots & \vdots \\
0 & \cdots & 0 & -1_n
\end{pmatrix} 
\end{equation}$
$\begin{equation}
h1 = 
\begin{pmatrix} 0_1 \\
0_2 \\
\vdots \\
0_n
\end{pmatrix} 
\end{equation}$

$\begin{equation}
G2 = 
\begin{pmatrix} 1 & 0 & \cdots & 0 \\
0 & 1 & \cdots & 0 \\
\vdots & \cdots & \ddots & \vdots \\
0 & \cdots & 0 & 1_n
\end{pmatrix} 
\end{equation}$
$\begin{equation}
h2 = 
\begin{pmatrix} (1+\Delta)C_1 \\
(1+\Delta)C_2 \\
\vdots \\
(1+\Delta)C_n
\end{pmatrix} 
\end{equation}$

$\begin{equation}
G3 = 
\begin{pmatrix} \xi_1^* & \xi_2^* & \cdots & \xi_n^*
\end{pmatrix} 
\end{equation}$ $\begin{equation}
h3 = 
\begin{pmatrix} C\sum_{i=1}^{\ell}\xi_i^*
\end{pmatrix} 
\end{equation}$

$\begin{equation}
G = 
\begin{pmatrix} G1 \\
G2 \\
G3
\end{pmatrix} 
\end{equation}$
and $\begin{equation}
h = 
\begin{pmatrix} h1 \\
h2 \\
h3
\end{pmatrix} 
\end{equation}$

In [ ]:
x = prob.X
y = prob.Y
C = prob.C

NUM = x.shape[0]
DIM = x.shape[1]

Ky = prob.yi_yj
Kx = prob.xi_xj
K = Ky*Kx
P = matrix(K)
q = matrix(-np.ones((NUM, 1)))
G1 = -np.eye(NUM)
G2 = np.eye(NUM)
G3 = xi_star.reshape(1,-1)
G = np.vstack((G1, G2))
G = np.vstack((G, G3))
G = matrix(G)
h1 = np.zeros(NUM).reshape(-1,1)
h2 = np.repeat(C, NUM).reshape(-1,1)
h3 = sum(xi_star)*C
h = np.vstack((h1, h2))
h = np.vstack((h, h3))
h = matrix(h)
A = matrix(y.reshape(1, -1))
b = matrix(np.zeros(1))
solvers.options['show_progress'] = False
sol = solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

In [ ]:
w = np.sum(alphas * y[:, None] * x, axis = 0)

bacond1 = (alphas > 1e-5)
bdcond2 = (alphas < C)
bcond = np.array([a and b for a, b in zip(bacond1, bdcond2)]).flatten()

yS = y[bcond]
xS = x[bcond]
aS = alphas[bcond]

sumTotal = 0
for s in range(len(yS)):
    innerTotal = 0
    for m in range(len(yS)):
        am = aS[m]
        ym = yS[m]
        xm_xs = prob.kernel(xS[m], xS[s])
        innerTotal += am*ym*xm_xs
    sumTotal += yS[s] - innerTotal

bias = sumTotal/len(yS)

svmdpsa_clf = classifier() # svmdpsa means svm delta plus simplified approach
svmdpsa_clf.w = w
svmdpsa_clf.b = bias
svmdpsa_clf.alphas = alphas
svmdpsa_clf.support_vectors = prob.X[bacond1.flatten()]

Let's check it out! The old classifier is on top and the new one below

In [ ]:
plot_margin(prob.X[y==1], prob.X[y==-1], svm_clf), plot_margin(prob.X[y==1], prob.X[y==-1], svmdpsa_clf)

Wait... after all that, they're the same?!?! What gives? Yeah... I'd have loved it if if was more impressive. But the only additional term is $\sum_{i=1}^{\ell}\alpha_i\xi_i^*\leq C\sum_{i=1}^{\ell}\xi_i^*$ and if your $\xi^*$, the margin violation in the privileged space, is 0. Well, it's not gonna make any difference. Let alone the fact that most $\alpha$'s are 0. We're looking for support vectors (non-zero $\alpha$'s) in $X$ space, that the data for them in the corresponding priviliged space $X^*$ is in breach of the margin. And given this toy data, that's not happening.

## SVM$_\Delta$+ - The not so simple approach

Well, the simplified approach wax a bit of a let down on the toy data. We might come back to it with some real data and see if it improves, but until then, let's look at the more complicated version and see if it manages to do any better. Here it is...

$$\begin{equation}
\begin{aligned}
\min_{\textbf{w},b,\textbf{w}^*,b^*} \quad \frac{1}{2}(||\textbf{w}||^2 + \gamma ||\textbf{w}^*||^2) & + C\sum_{i=1}^{\ell} [y_i((\textbf{w}^* \cdot \textbf{x}^*_i) + b^*) + \zeta_i] + \Delta C \sum_{i=1}^{\ell}\zeta_i \\
\textrm{subject to} & \quad y_i(\textbf{w} \cdot \textbf{x}_i+b)\geq 1 - y_i((\textbf{w}^* \cdot \textbf{x}^*_i) + b^*)-\zeta_i \\
\textrm{and} & \quad y_i((\textbf{w}^* \cdot \textbf{x}^*_i) + b^*) + \zeta_i \geq 0 \\
\textrm{and} & \quad \zeta_i \geq 0 
\end{aligned}
\end{equation}$$

Crumbs, that looks complicated. But it's similar in form to the SVM. We have a term that we want to minimise subject to a few things. Let's do what we did with the SVM and get the dual form and see if we can follow the steps we did before.

$$\begin{equation}
\begin{aligned}
\max_{\alpha, (C-\beta)} \sum_{i=1}^{\ell}\alpha_i - \frac{1}{2}\sum_{i,j=1}^{\ell}\alpha_i \alpha_j y_i y_j (\textbf{x}_i \cdot \textbf{x}_j) - & \frac{1}{2\gamma}\sum_{i, j = 1}^{\ell}y_i y_j(C-\beta_i - \alpha_i)(C-\beta_j - \alpha_j)(\textbf{x}_i^* \cdot \textbf{x}_j^*) \\
\textrm{subject to} & \quad \sum_{i=1}^{\ell} y_i \alpha_i = 0 \\
\textrm{and} & \quad \sum_{i=1}^{\ell} y_i C-\beta_i = 0 \\
\textrm{and} & \quad 0 \leq C-\beta_i \leq C \\
\textrm{and} & \quad 0 \leq \alpha_i \leq C-\beta_i +\Delta C
\end{aligned}
\end{equation}$$

Let's neaten it up a bit and say $\beta-C = \delta$

$$\begin{equation}
\begin{aligned}
\max_{\alpha, \delta} \sum_{i=1}^{\ell}\alpha_i - \frac{1}{2}\sum_{i,j=1}^{\ell}\alpha_i \alpha_j y_i y_j (\textbf{x}_i \cdot \textbf{x}_j) - & \frac{1}{2\gamma}\sum_{i, j = 1}^{\ell}y_i y_j(\delta_i - \alpha_i)(\delta_j - \alpha_j)(\textbf{x}_i^* \cdot \textbf{x}_j^*) \\
\textrm{subject to} & \quad \sum_{i=1}^{\ell} y_i \alpha_i = 0 \\
\textrm{and} & \quad \sum_{i=1}^{\ell} y_i \delta_i = 0 \\
\textrm{and} & \quad 0 \leq \delta_i \leq C \\
\textrm{and} & \quad 0 \leq \alpha_i \leq \delta_i +\Delta C
\end{aligned}
\end{equation}$$

So looking at the constraints, goven that we've done this kind of thing before, they shouldn't be too much of a problem. Yes, there's now an extra variable, $\delta$, but we can deal with that. The more scary looking one is the top line. Unfotunately I'm not an expert in just wizardly turning that into one matrix. However, [this paper](http://www.cse.psu.edu/~zbc102/files/svm_plus_technical_report_15.pdf) solves a problem that looks very similar.
$$\begin{equation}
\begin{aligned}
\max_{\alpha, \delta} \sum_{i=1}^{\ell}\alpha_i - \frac{1}{2}\sum_{i,j=1}^{\ell}\alpha_i \alpha_j y_i y_j (\textbf{x}_i \cdot \textbf{x}_j) - & \frac{\gamma}{2}\sum_{i, j = 1}^{\ell}y_i y_j(\alpha_i- \delta_i)(\alpha_j - \delta_j)(\textbf{x}_i^* \cdot \textbf{x}_j^*) \\
\textrm{subject to} & \quad \sum_{i=1}^{\ell} y_i \alpha_i = 0 \\
\textrm{and} & \quad \sum_{i=1}^{\ell} y_i \delta_i = 0 \\
\textrm{and} & \quad 0 \leq \delta_i \leq C_i \\
\textrm{and} & \quad 0 \leq \alpha_i \leq \kappa C_i
\end{aligned}
\end{equation}$$

Let's look at the differences 
1. The placing of $\gamma$, well that's no biggie, it will still have the same effect. 
2. We talk about $(\alpha_i - \delta_i)$, the other paper solves $(\delta_i - \alpha_i)$. Now this could be a bit of a problem, we'll talk about it in a bit.
3. They talk about $\kappa C$, we're looking at $\delta_i + \Delta C$. That's easy enough, we can just change a contstraint a little bit.
4. They talk about $C_i$. This is a bit confusing as $C$ is a constant. I'm hoping this is a typo and nothing more profound.

Let's get the constraints out of the way, because we can do them easily. First off, the inequalities.

G1 and h1 $-\delta_i \leq 0$

G2 and h2 $\delta_i \leq C$

G3 and h3 $-\alpha_i \leq 0$

G4 and h4 $\alpha_i \leq \delta_i + \Delta C$ which we'll jig about to be $\alpha_i - \delta_i \leq  \Delta C$

The thing to remember is that these are being multiplied by two variables, as such we need to remember that we will be multiplying $G$ by $\begin{equation}
\begin{pmatrix} \alpha \\
\delta
\end{pmatrix} 
\end{equation}$

$\begin{equation}
G1 = 
\begin{pmatrix} 0 & 0 & \cdots & 0_{1,n} & -1_{1,n+1} & 0 & \cdots & 0_{1,2n} \\
0 & 0 & \cdots & 0_{2,n} & 0_{2,n+1} & -1 & \cdots & 0_{2,2n} \\
\vdots & \cdots & \ddots & \vdots & \vdots & \cdots & \ddots & \vdots \\
0_{n,1} & \cdots & 0 & 0_{n,n} & 0_{n, n+1} & \cdots & 0 & -1_{n,2n}
\end{pmatrix} 
\end{equation}$
$\begin{equation}
h1 = 
\begin{pmatrix} 0_1 \\
0_2 \\
\vdots \\
0_n
\end{pmatrix} 
\end{equation}$

$\begin{equation}
G2 = 
\begin{pmatrix} 0 & 0 & \cdots & 0_{1,n} & 1_{1,n+1} & 0 & \cdots & 0_{1,2n} \\
0 & 0 & \cdots & 0_{2,n} & 0_{2,n+1} & 1 & \cdots & 0_{2,2n} \\
\vdots & \cdots & \ddots & \vdots & \vdots & \cdots & \ddots & \vdots \\
0_{n,1} & \cdots & 0 & 0_{n,n} & 0_{n, n+1} & \cdots & 0 & 1_{n,2n}
\end{pmatrix} 
\end{equation}$
$\begin{equation}
h2 = 
\begin{pmatrix} C_1 \\
C_2 \\
\vdots \\
C_n
\end{pmatrix} 
\end{equation}$

$\begin{equation}
G3 = 
\begin{pmatrix} -1_{1,1} & 0 & \cdots & 0_{1,n} & 0_{1,n+1} & 0 & \cdots & 0_{1,2n} \\
0_{2,1} & -1 & \cdots & 0_{2,n} & 0_{2,n+1} & 0 & \cdots & 0_{2,2n} \\
\vdots & \cdots & \ddots & \vdots & \vdots & \cdots & \ddots & \vdots \\
0_{n, 1} & \cdots & 0 & -1_{n,n} & 0_{n,n+1} & \cdots & 0 & 0_{n,2n}
\end{pmatrix} 
\end{equation}$
$\begin{equation}
h3 = 
\begin{pmatrix} 0_1 \\
0_2 \\
\vdots \\
0_n
\end{pmatrix} 
\end{equation}$

$\begin{equation}
G4 = 
\begin{pmatrix} 1_{1,1} & 0 & \cdots & 0_{1,n} & -1_{1,n+1} & 0 & \cdots & 0_{1,2n} \\
0_{2,1} & 1 & \cdots & 0_{2,n} & 0_{2,n+1} & -1 & \cdots & 0_{2,2n} \\
\vdots & \cdots & \ddots & \vdots & \vdots & \cdots & \ddots & \vdots \\
0_{n, 1} & \cdots & 0 & 1_{n,n} & 0_{n,n+1} & \cdots & 0 & -1_{n,2n}
\end{pmatrix} 
\end{equation}$
$\begin{equation}
h4 = 
\begin{pmatrix} \Delta C \\
\Delta C \\
\vdots \\
\Delta C
\end{pmatrix} 
\end{equation}$

And as before $\begin{equation}
G = 
\begin{pmatrix} G1 \\
G2 \\
G3 \\
G4
\end{pmatrix} 
\end{equation}$
and $\begin{equation}
h = 
\begin{pmatrix} h1 \\
h2 \\
h3 \\
h4
\end{pmatrix} 
\end{equation}$

Now, the equalities!

A1 = $\sum_{i=1}^{\ell}y_i\alpha_i$ b1 = $0$
A2 = $\sum_{i=1}^{\ell}y_i\delta_i$ b2 = 0

$\begin{equation}
A1 = 
\begin{pmatrix} y_1 & y_2 & \cdots & y_n & 0_{n+1} & 0_{n+2} & \cdots & 0_{2n}
\end{pmatrix} 
\end{equation}$ $\begin{equation}
b1 = 
\begin{pmatrix} 0
\end{pmatrix} 
\end{equation}$

$\begin{equation}
A2 = 
\begin{pmatrix} 0_{1} & 0_{2} & \cdots & 0_{n} & y_{n+1} & y_{n+2} & \cdots & y_{2n}
\end{pmatrix} 
\end{equation}$ $\begin{equation}
b2 = 
\begin{pmatrix} 0
\end{pmatrix} 
\end{equation}$

Then stack as with inequalities. 
$\begin{equation}
A = 
\begin{pmatrix} A1 \\
A2 
\end{pmatrix} 
\end{equation}$
and $\begin{equation}
b = 
\begin{pmatrix} b1 \\
b2 
\end{pmatrix} 
\end{equation}$

Yeah, you could've probably done them yourself. But then I wouldn't get to put off this part. In the paper that solves a similar problem, they call their $P$ and $q$, $H$ and $f$. Fine. The $f$ is easy, we're doing a sum over all the negative $\alpha$'s, so $\begin{equation}
f = q = 
\begin{pmatrix} -1_1 & -1_2 & \cdots & -1_n & 0_{n+1} & \cdots & 0_{2n}
\end{pmatrix} 
\end{equation}$

The paper gives $\begin{equation}
H = 
\begin{pmatrix} (\textbf{x}_i,\textbf{x}_j)y_iy_j + \gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j & -\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j \\
-\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j & +\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j
\end{pmatrix} 
\end{equation}$

When I've got a bit more time I'll do a proper expansion of H, just know that it's a 2n by 2n matrix. The bit that's confusing me is that we need to go from the optimisation problem to H. Maybe we can work it backwards?

$\begin{equation}
\begin{pmatrix} \alpha & \delta \end{pmatrix} \times
\begin{pmatrix} (\textbf{x}_i,\textbf{x}_j)y_iy_j + \gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j & -\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j \\
-\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j & +\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j
\end{pmatrix} \times 
\begin{pmatrix} \alpha \\
\delta
\end{pmatrix}
\end{equation}$

$\begin{pmatrix} (\alpha((\textbf{x}_i,\textbf{x}_j)y_iy_j + \gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j))+(\delta(-\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j)) &
(\alpha(-\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j))+ 
(\delta(\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j))
\end{pmatrix} \times 
\begin{pmatrix} \alpha \\
\delta
\end{pmatrix}$

$\begin{pmatrix}
\alpha(\alpha((\textbf{x}_i,\textbf{x}_j)y_iy_j + \gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j) +
(\delta(-\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j))) & + &
\delta(\alpha(-\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j)+ 
(\delta(\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j)))
\end{pmatrix}$

$\begin{pmatrix}
\alpha\alpha((\textbf{x}_i,\textbf{x}_j)y_iy_j + \gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j) +
\alpha\delta(-\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j) & + &
\delta\alpha(-\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j)+ 
\delta\delta(\gamma(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j)
\end{pmatrix}$

$\begin{pmatrix}
\alpha\alpha(\textbf{x}_i,\textbf{x}_j)y_iy_j + \gamma\alpha\alpha(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j
-\gamma\alpha\delta(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j
-\gamma\delta\alpha(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j+ 
\gamma\delta\delta(\textbf{x}^*_i,\textbf{x}^*_j)y_iy_j
\end{pmatrix}$

$\begin{pmatrix}
\alpha\alpha y_iy_j(\textbf{x}_i,\textbf{x}_j) + \gamma y_iy_j(\textbf{x}^*_i,\textbf{x}^*_j)\alpha\alpha
-\gamma y_iy_j(\textbf{x}^*_i,\textbf{x}^*_j)\alpha\delta
-\gamma y_iy_j(\textbf{x}^*_i,\textbf{x}^*_j)\alpha\delta+ 
\gamma y_iy_j(\textbf{x}^*_i,\textbf{x}^*_j)\delta\delta
\end{pmatrix}$

$\begin{pmatrix}
\alpha\alpha y_iy_j(\textbf{x}_i,\textbf{x}_j) + \gamma y_iy_j(\textbf{x}^*_i,\textbf{x}^*_j)
(\alpha\alpha-\alpha\delta-\alpha\delta+\delta\delta)
\end{pmatrix}$

I hope you've followed those steps. I'll try to come back and annotate them a bit more, but now we've got to an interesting part. $(\alpha\alpha - \alpha\delta - \alpha\delta + \delta\delta)$. If we factorise this, we get... $(\alpha-\delta)(\alpha-\delta)$. Which is great. But we can also get $(\delta-\alpha)(\delta-\alpha)$. Which is really sweet. The same $H$ or $P$ matrix works for both the problems. Awesome! So let's populate this thing.

In [ ]:
kernel = prob.kernel
C = prob.C

L = prob.num
M = prob.dimensions

x = prob.X
y = prob.Y

gamma = prob.gamma
delta = prob.delta

H11 = (prob.xi_xj * prob.yi_yj) + gamma*(prob.xstari_xstarj * prob.yi_yj)
H12 = -gamma*(prob.xstari_xstarj * prob.yi_yj)
H22 = gamma*(prob.xstari_xstarj * prob.yi_yj)
H1 = np.hstack((H11, H12))
H2 = np.hstack((H12, H22))
H = np.vstack((H1, H2))

f = np.hstack((np.repeat(-1, L),np.zeros(L)))

positiveEye = np.eye(L, dtype='d')
negativeEye = -np.eye(L, dtype='d')
zeros = np.zeros((L, L))
g1 = np.hstack((zeros, negativeEye))
g2 = np.hstack((zeros, positiveEye))
g3 = np.hstack((negativeEye, zeros))
g4 = np.hstack((positiveEye, negativeEye))

G = np.vstack((g1,g2))
G = np.vstack((G,g3))
G = np.vstack((G,g4))

h1 = np.zeros(((L),1))
h2 = np.repeat(C, (L)).reshape(-1,1)
h2 = np.vstack((h1, h2))
h3 = np.vstack((h2, h1))
h4 = np.repeat((delta*C), L).reshape(-1,1)
h = np.vstack((h3, h4))

Aeq1 = np.hstack((prob.Y, np.zeros(L)))
Aeq2 = np.hstack((np.zeros(L), prob.Y))
Aeq = np.vstack((Aeq1, Aeq2))

beq = np.zeros(2)
beq = beq.reshape(-1,1)

P = matrix(H, tc='d')
q = matrix(f, tc='d')
G = matrix(G, tc='d')
h = matrix(h, tc='d')
A = matrix(Aeq, tc='d')
b = matrix(beq, tc='d')

solvers.options['show_progress'] = False
sol = solvers.qp(P, q, G, h, A, b)
alphasAndDeltas = np.array(sol['x'])
alphas = alphasAndDeltas[:L]
deltas = alphasAndDeltas[L:]

And then we get $\textbf{w}$ in the same way we always have done

In [ ]:
w = np.sum(alphas * y[:, None] * x, axis = 0)

The bias however, is slightly different. Instead of $b = \frac{\sum_{s=1}^{S}(y_s-\sum_{m=1}^{S}\alpha_my_m(\textbf{x}_m\cdot\textbf{x}_s))}{\textrm{Number of S}}$ we have $b=1-y_k[\sum_{i=1}^{\ell}\alpha_i(\textbf{x}_i,\textbf{x}_k)]$ where $x_i\neq 0$, $\delta_k \neq C$ and $\alpha_k \neq 0$.

In [ ]:
bacond = (alphas > 1e-5)
bdcond = (deltas < C)
bxcond = (x != 0)

bxcond2 = list(range(0, L))
index = 0
for dataPoint in bxcond:
    for point in dataPoint:
        if point == False:
            bxcond2[index] = False
            break
        bxcond2[index] = True
    index += 1

bcond = np.array([a and b for a, b in zip(bacond, bdcond)]).flatten()
bcond = np.array([a and b for a, b in zip(bcond, bxcond2)]).flatten()
yK = y[bcond]
xK = x[bcond]

b = []
for k in range(len(xK)):
    b.append(1-yK[k]*np.dot(w, xK[k]))

Ok, so I'm not entirely happy. The there are multiple posints where the $x_i\neq 0$, $\delta_k \neq C$ and $\alpha_k \neq 0$ contraints are valid. So, I'm going to just take the average (mean) point.

In [ ]:
bias = (1- (sum(b) / len(b)))

In [ ]:
svmdp_clf = classifier() # svmdpsa means svm delta plus simplified approach
svmdp_clf.w = w
svmdp_clf.b = bias
svmdp_clf.alphas = alphas
svmdp_clf.support_vectors = prob.X[bacond.flatten()]

Let's see what it looks like shall we? The originl classifier is on top, the SVM$_\Delta$+ below.

In [ ]:
plot_margin(prob.X[y==1], prob.X[y==-1], svm_clf), plot_margin(prob.X[y==1], prob.X[y==-1], svmdp_clf)

Whoah!!!! What's happened? Well, the data is generated randomly, so what you're seeing isn't the same as what I am. But you should see that the support vectors (the ones with green circles around them) are a little different and the boundary's moved a bit. Why's that? Good question. I'll have a think about it and try to report back.